In [2]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

# llm = OpenAI()
llm = OpenAI(model_name="gpt-3.5-turbo-1106")
chat = ChatOpenAI()

a = llm.predict("How many planets are there?")
b = chat.predict("How many planets are there?")

a, b

/Users/jjack/Documents/fullstack-gpt/env/lib/python3.9/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/jjack/Documents/fullstack-gpt/env/lib/python3.9/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


('There are eight recognized planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. Additionally, there is a dwarf planet called Pluto that was reclassified as a "plutoid" or "trans-Neptunian object" by the International Astronomical Union in 2006.',
 'As of now, there are eight confirmed planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. However, there is ongoing debate among scientists regarding the classification of Pluto as a planet. Some consider it as a dwarf planet, while others still consider it as the ninth planet.')

In [5]:
chat = ChatOpenAI(temperature=0.1)

In [7]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(
    content="You are a geography expert. And you only reply in Italian."
),
AIMessage(content="Chao, mi chiamo Paolo!"),
HumanMessage(content="What is the distance between Mexico and Thailand. Also, what is your name?")
]

chat.predict_messages(messages)

AIMessage(content='Ciao! Il mio nome è Paolo. La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri.')

In [11]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate
chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}.",
)

prompt = template.format(country_a="Mexico", country_b="Thailand")


chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles).'

In [13]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Chao, mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_a}. Also, what is your name?")
])

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand",
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σου! Η απόσταση μεταξύ Μεξικού και Μεξικού είναι μηδέν, καθώς αναφέρεσαι στην ίδια τοποθεσία. Το όνομά μου είναι Σωκράτης.')

In [15]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser): # expand

    def parse(self, text): # necc
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
p = CommaOutputParser()

p.parse("Hello, how, are, you")


['Hello', 'how', 'are', 'you']

In [23]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else."),
    ("human", "{question}"),
])

prompt = template.format_messages(max_items=10, question= "What are the colors?")

result = chat.predict_messages(prompt)

p = CommaOutputParser()
p.parse(result.content)

['red',
 'orange',
 'yellow',
 'green',
 'blue',
 'indigo',
 'violet',
 'black',
 'white',
 'gray']

In [24]:
chat.predict_messages(prompt)

AIMessage(content='red, orange, yellow, green, blue, indigo, violet, black, white, gray')

In [25]:
# chain
chain = template | chat | CommaOutputParser()

# run chain
chain.invoke({
    "max_items": 5,
    "question": "What are the poketmons?"
})

['pikachu', 'charizard', 'bulbasaur', 'squirtle', 'jigglypuff']